In [1]:
import requests 
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import numpy as np   
import pandas as pd  
import time          
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

In [32]:
page_link = 'https://mtgsale.ru/home/search-results?Name=&Lang=Any&Type=Any&Color=Any&Rarity=Any&Foil=Any&minP=3&maxP=34400&Page=5&Sort=name&CardSet='
# page_link ='https://scryfall.com/card/vow/249/torens-fist-of-the-angels'

In [33]:
UserAgent().chrome

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2224.3 Safari/537.36'

In [34]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [35]:
html = response.content

In [36]:
soup = BeautifulSoup(html, 'html.parser')

In [45]:
obj = soup.find('a', attrs = {'class':'tnamec'})
obj.attrs['href']

<td class="flexbox">
<a href="https://scryfall.com/sets/gn3">
<svg aria-hidden="true" focusable="false" viewbox="0 0 38.44 36.63" xmlns="http://www.w3.org/2000/svg"><path d="M28.16 25.42L38.44 18 35.1 7.71l-6.4-.07-.12 4.43L24.71 0H13.89l-2.05 6.07 4.18 1.47L3.34 7.5 0 17.79l5.13 3.82 2.7-3.52-3.96 12.05 8.76 6.35 5.22-3.7-2.51-3.65 10.23 7.49 8.75-6.36-1.91-6.11-4.25 1.26z"></path></svg>

                Game Night 2022 <small>GN3</small>
</a></td>

In [8]:
mtg_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['tnamec'])

In [9]:
mtg_links = [link.attrs['href'] for link in mtg_links]

In [26]:
def getPageLinks(cite, page_number):
    """
        Возвращает список ссылок на карты mtg, полученный с текущей страницы
        
        page_number: int/string
            номер страницы для парсинга
    """
    # составляем ссылку на страницу поиска
    page_link = cite.format(page_number)
    
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    # запрашиваем данные по ней
    response = session.get(page_link, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return [] 
    
    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    # наконец, ищем ссылки и очищаем их от ненужных тэгов
    mtg_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['tnamec'])
    mtg_links = ['https://mtgsale.ru' + link.attrs['href'] for link in mtg_links if '/singles/' in link.attrs['href'] ]
    
    
    return mtg_links

In [27]:
mtg_links = getPageLinks('https://mtgsale.ru/home/search-results?Page={}',4)
mtg_links[:2]

['https://mtgsale.ru/item/singles/uh_eng/ach-hans-run_116',
 'https://mtgsale.ru/item/singles/ust_eng/rumors-of-my-death---_65']

In [12]:
page = 'https://mtgsale.ru/item/singles/m15_rus/act-on-impulse_126'
page = 'https://mtgsale.ru/item/singles/afr_eng/2-mace_1'
# page = 'https://mtgsale.ru/item/singles/prerp_rus/adeliz-the-cinder-wind_155'
# page = 'https://mtgsale.ru/item/singles/dtk_rus/aerie-bowmasters_170'
response = requests.get(page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

In [13]:
compl_props = [['i','lang'], ['span','nabor']]
lots_props = [['a','tnamec', 0], ['span','smallfont', 0],  
              ['p','foil', 1], ['p','sost', 1], ['p','redkost', 1], ['p','pprice', 1], ['p','colvo', 1]]

In [14]:
# card = [x for x in soup.findAll('p', attrs={'class':'regular'})]

In [15]:
def get_descr(smallsoup):
    card_descriprion = []
    card_abilities = []
    card = [x for x in smallsoup.findAll('p', attrs={'class':'regular'})]
    if str(card[0]).find('<br/>') > -1:
        card_abilities = [[str(x).replace("<br/>", "|").strip('<p class="regular">\r\n').split('|')[0]] for x in card]
#         Полное описание карты пока не нужно
#         card_descriprion = [str(x).replace("<br/>", "|").strip('\r\n            </p>').split('|')[1] for x in card]
    else:
        pass
#         Полное описание карты пока не нужно
#         card_descriprion = [x.text.strip() for x in card]
    return card_abilities, #card_descriprion

In [16]:
def get_detailed(small_soup, pr0, pr1):
    if pr1 == 'lang':
        clean_attrs = [x.get('title') for x in soup.find_all(pr0, attrs={'class': lambda e: e.startswith('flag') if e else False})]
    else:
        clean_attrs = [x.get('title') for x in soup.find_all('span', attrs={'class': lambda e: e.startswith('n-') if e else False})]
    return clean_attrs

In [17]:
def getProperties(soup):
    """
        Возвращает список с описанием
    """
    clean_properties = []
    card_description = []
    # название - идёт с самым большим заголовком h1, легко найти
    card_description = get_descr(soup)
    for compl_prop in compl_props:
        attrs = get_detailed(soup, compl_prop[0], compl_prop[1])
        clean_properties.append(attrs)
    for prop in lots_props:
        attrs = [x.text.strip() for x in soup.findAll(prop[0], attrs={'class':lambda e: e.startswith(prop[1]) if e else False})]
        clean_properties.append(attrs[prop[2]:])
    if len(card_description[0]) == 0: 
        card_description = [[''],['']]
    for cd in card_description:
#         clean_properties.append([val for val in cd for _ in range(len(clean_properties[0]))])
        for cd_small in cd:
            clean_properties.append(np.repeat(cd_small, len(clean_properties[0])))
    
    result = np.array(clean_properties, dtype=object).T

    return result

In [18]:
mtg_links[2]

'https://mtgsale.ru/item/singles/afr_eng/2-mace_1'

In [19]:
rows = getProperties(soup)
rows

array([['Английский', 'Adventures in the Forgotten Realms', '+2 Mace',
        '', '', 'NM', 'Обычная', '8 ₽', '0 шт.',
        'Equipped creature gets +2/+2.'],
       ['Английский', 'Adventures in the Forgotten Realms', '+2 Mace',
        '', 'Фойл', 'NM', 'Обычная', '18 ₽', '2 шт.',
        'Equipped creature gets +2/+2.']], dtype=object)

In [20]:
# final_df = pd.DataFrame(columns=['Language', 'Set', 'Name1', 'Name2', 'Foil',
#                                  'Quality', 'Rarity', 'Price', 'Qty', 'Special1', 'Special2'])
# df_row = 0
# final_df.loc[df_row] = rows[0]

In [21]:
final_df = pd.DataFrame(columns=['Language', 'Set', 'Name1', 'Name2', 'Foil',
                                 'Quality', 'Rarity', 'Price', 'Qty', 'Special1', 'Special2'])
df_row = 0
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)


for page in tqdm(mtg_links):
    response = session.get(page, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    rows = getProperties(soup)
    for parsed_row in rows:
#         final_df = pd.DataFrame(columns=['Language', 'Set', 'Name1', 'Name2', 'Foil',
#                                  'Quality', 'Rarity', 'Price', 'Qty', 'Special1', 'Special2'])
        if len(parsed_row) < 11:
            parsed_row = np.insert(parsed_row, 9, '')
        final_df.loc[df_row] = parsed_row
        df_row += 1
final_df

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  7.41it/s]


,Language,Set,Name1,Name2,Foil,Quality,Rarity,Price,Qty,Special1,Special2
0,Английский,Unhinged,"""Ach! Hans, Run!""",,,NM,Редкая,370 ₽,1 шт.,,
1,Английский,Unstable,"""Rumors of My Death . . .""",,,NM,Необычная,7 ₽,13 шт.,,
2,Английский,Adventures in the Forgotten Realms,+2 Mace,,,NM,Обычная,8 ₽,0 шт.,,Equipped creature gets +2/+2.
3,Английский,Adventures in the Forgotten Realms,+2 Mace,,Фойл,NM,Обычная,18 ₽,2 шт.,,Equipped creature gets +2/+2.
4,Русский,Adventures in the Forgotten Realms,+2 Mace [RUS],+2 Mace,,NM,Обычная,8 ₽,9 шт.,,Equipped creature gets +2/+2.
5,Русский,Adventures in the Forgotten Realms,+2 Mace [RUS],+2 Mace,Фойл,NM,Обычная,22 ₽,0 шт.,,Equipped creature gets +2/+2.
6,Русский,Eldritch Moon,Променять Мудрость,Abandon Reason,,NM,Необычная,14 ₽,3 шт.,Не более двух целевых существ получают по +1/+...,Up to two target creatures each get +1/+0 and ...
7,Русский,Eldritch Moon,Променять Мудрость,Abandon Reason,,SP,Необычная,13 ₽,0 шт.,Не более двух целевых существ получают по +1/+...,Up to two target creatures each get +1/+0 and ...
8,Русский,Eldritch Moon,Променять Мудрость,Abandon Reason,,MP,Необычная,11 ₽,0 шт.,Не более двух целевых существ получают по +1/+...,Up to two target creatures each get +1/+0 and ...
9,Русский,Eldritch Moon,Променять Мудрость,Abandon Reason,Фойл,NM,Необычная,45 ₽,0 шт.,Не более двух целевых существ получают по +1/+...,Up to two target creatures each get +1/+0 and ...


In [23]:
final_df = pd.DataFrame(columns=['Language', 'Set', 'Name1', 'Name2', 'Foil',
                                 'Quality', 'Rarity', 'Price', 'Qty', 'Special1', 'Special2'])
df_row = 0

In [30]:
for page_number in tqdm(range(3000, 3734), desc='Pages'):
    # собрали хрефы с текущей страницы
    mtg_links = getPageLinks('https://mtgsale.ru/home/search-results?Page={}', page_number)  
    
    session = requests.Session()
    retry = Retry(connect=5, backoff_factor=1.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    for mtg_links in mtg_links:
        # иногда с первого раза страничка не парсится
        for i in range(3):
            try:
                # пытаемся собрать по мему немного даты
                
                response = session.get(mtg_links, headers={'User-Agent': UserAgent().chrome})
                
#                 response = requests.get(page, headers={'User-Agent': UserAgent().chrome}) 
                html = response.content
                soup = BeautifulSoup(html,'html.parser')
                rows = getProperties(soup)
                for parsed_row in rows:
                    if len(parsed_row) < 11:
                        parsed_row = np.insert(parsed_row, 9, '')
                    final_df.loc[df_row] = parsed_row
                    df_row += 1
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', mtg_links)
                continue
final_df

Pages: 100%|███████████████████████████████████████████████████████████████████████| 734/734 [1:55:30<00:00,  9.44s/it]


,Language,Set,Name1,Name2,Foil,Quality,Rarity,Price,Qty,Special1,Special2
0,Английский,Unhinged,"""Ach! Hans, Run!""",,,NM,Редкая,370 ₽,1 шт.,,
1,Английский,Unstable,"""Rumors of My Death . . .""",,,NM,Необычная,7 ₽,13 шт.,,
2,Английский,Adventures in the Forgotten Realms,+2 Mace,,,NM,Обычная,8 ₽,0 шт.,,Equipped creature gets +2/+2.
3,Английский,Adventures in the Forgotten Realms,+2 Mace,,Фойл,NM,Обычная,18 ₽,2 шт.,,Equipped creature gets +2/+2.
4,Русский,Adventures in the Forgotten Realms,+2 Mace [RUS],+2 Mace,,NM,Обычная,8 ₽,9 шт.,,Equipped creature gets +2/+2.
...,...,...,...,...,...,...,...,...,...,...,...
219610,Английский,Kaladesh,Æther Tradewinds,Эфирный Пассат,Фойл,NM,Обычная,19 ₽,0 шт.,,
219611,Английский,Darksteel,Æther Vial,,,NM,Необычная,1620 ₽,0 шт.,,"At the beginning of your upkeep, you may put a..."
219612,Английский,Darksteel,Æther Vial,,,SP,Необычная,1470 ₽,0 шт.,,"At the beginning of your upkeep, you may put a..."
219613,Английский,Darksteel,Æther Vial,,,MP,Необычная,1310 ₽,0 шт.,,"At the beginning of your upkeep, you may put a..."


In [31]:
final_df.to_csv('parsed_data3.csv') 